In [31]:
import numpy as np
import pandas as pd
from random import sample
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly import graph_objects as go
from collections import defaultdict

In [95]:
data = pd.read_csv('../program/program.csv', sep=';')
scores = defaultdict(int)
data.head()

,number,text
0,1,Вероятносное пространство. Аксиоматика Колмого...
1,2,Вероятносные меры на R. Функция распределения ...
2,3,Классификация вероятносных мер и функций распр...


In [167]:


def get_layout(title='', height=500):
    standart_layout = go.Layout({
        'height': height,
        'width': 600,
        'title': title,
        'template': 'plotly_white'
    })
    
    return standart_layout


def get_random_ticket():
    return sample([i for i in range(data.shape[0])], k=1)[0]

CUR_TICKET = 0
STATISTIC = defaultdict(int)
G_CLICKS = 0
B_CLICKS = 0
S_CLICKS = 0
for i in range(data.shape[0]):
    STATISTIC[i] = [0, 0]

# Front

In [168]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Start'),
    
    html.Div([dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Распределение по билетам')),
                       id='main_hist',
                       style={'display': 'inline-block'}),
              dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Прогресс')),
                       id='main_prog',
                       style={'display': 'inline-block'})
             ],
             style={'display': 'inline-block'}
    ),
    html.Hr(),
    html.H3(id='display_ticket', 
             children="Нажми start",
             style={'text-align': 'center'}
            ),
    html.Div(
        [html.Button('Start', id='start_button', value='Start')],
        style={'content-align': 'center'}
    ),
    html.Div([dcc.Graph(figure=go.Figure(data=[], layout=get_layout('Статистика по билету')), id='ticket_stat')],
             style={'display': 'inline-block', 'centering': 'middle'},
             ),
    html.Hr(),
    html.Div([
        html.Button('Ok', id='good_button'),
        html.Span(),
        html.Button('Не Ok', id='bad_button')
    ], 
    style={'display': 'inline-block', 'centering': 'middle'}
    )
    
])
    

# Back

In [169]:
@app.callback(
    Output('display_ticket', 'children'),
    [Input('start_button', 'n_clicks')]
)
def display_text(n_clicks):
    if n_clicks is None:
        return u"Нажми start"
    else:
        global CUR_TICKET
        CUR_TICKET = get_random_ticket()
        return data['text'][CUR_TICKET]

@app.callback(
    Output('ticket_stat', 'figure'),
    [Input('start_button', 'n_clicks'),
     Input('good_button', 'n_clicks'), 
     Input('bad_button', 'n_clicks')]
)
def get_statistic(n_clicks, g_n_clicks=None, b_n_clicks=None):
    
    global G_CLICKS, B_CLICKS, STATISTIC, CUR_TICKET, S_CLICKS
    if n_clicks is not None:
        if n_clicks != S_CLICKS:
            CUR_TICKET = get_random_ticket()
            S_CLICKS += 1
    if g_n_clicks is not None:
        if b_n_clicks is not None:
            if G_CLICKS != g_n_clicks:
                to_update = STATISTIC[CUR_TICKET]
                to_update[0] += 1
                STATISTIC[CUR_TICKET] = to_update
                G_CLICKS += 1
                print(to_update)
                
            if B_CLICKS != b_n_clicks:
                to_update = STATISTIC[CUR_TICKET]
                to_update[1] += 1
                print(to_update)
                STATISTIC[CUR_TICKET] = to_update
                B_CLICKS += 1
                
        if G_CLICKS != g_n_clicks:
            to_update = STATISTIC[CUR_TICKET]
            to_update[0] += 1
            print(to_update)
            STATISTIC[CUR_TICKET] = to_update
            G_CLICKS += 1
            
    
    if b_n_clicks is not None:   
        if B_CLICKS != b_n_clicks:
            to_update = STATISTIC[CUR_TICKET]
            to_update[1] += 1
            print(to_update)
            STATISTIC[CUR_TICKET] = to_update
            B_CLICKS += 1
    
    x = ['Правильно', 'Неправильно']           
    y = STATISTIC[CUR_TICKET]
    bar = go.Bar({
        'x': x,
        'y': y,
        'marker': {
            'color': ['green', 'red']
        }
    })
    return go.Figure(data=[bar], layout=get_layout('Статистика по билету'))
    


In [170]:
app.server.run(host='localhost', port=8899)

 * Running on http://localhost:8899/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563475972 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563475972 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563475972 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_core_components/plotly-1.48.3.min.js?v=1.0.0&m=1563474247 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563474247 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:23] "GET /_dash-component-suites/dash_core_components/dash_core_components.min.js?v=1.0.0&m=1563474247 HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep

[1, 0]


127.0.0.1 - - [14/Sep/2019 16:48:29] "POST /_dash-update-component HTTP/1.1" 200 -


[1, 1]


127.0.0.1 - - [14/Sep/2019 16:48:30] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [14/Sep/2019 16:48:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:48:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.

[3, 1]


127.0.0.1 - - [14/Sep/2019 16:49:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Sep/2019 16:49:11] "POST /_dash-update-component HTTP/1.1" 200 -


[1, 1]


127.0.0.1 - - [14/Sep/2019 16:49:14] "POST /_dash-update-component HTTP/1.1" 200 -


defaultdict(int, {0: [1, 1], 1: [0, 0], 2: [3, 1]})